# El Output que espera FS para el Dashboard

In [1]:
{
    "title": "Evento 4",
    "description": "Descripción 4",
    "speacker": "Orador 4",
    "day": "20",
    "month": "Diciembre",
    "attendees": {
        "registered": 1,
        "confirmed": 1,
        "present": 1
    },
    "capacity": 30,
    "entry_exit": [
        {
            "id": "Entradas",
            "data": [
                {
                    "x": "10:00",
                    "y": 1
                }
            ]
        },
        {
            "id": "Salidas",
            "data": [
                {
                    "x": "12:00",
                    "y": 1
                }
            ]
        }
    ],
    "nationality": [
        {
            "country": "País 4",
            "userCount": 1
        }
    ],
    "type": [
        {
            "id": 4,
            "typeName": "Sector X",
            "typeCount": 1
        }
    ]
}

{'title': 'Evento 4',
 'description': 'Descripción 4',
 'speacker': 'Orador 4',
 'day': '20',
 'month': 'Diciembre',
 'attendees': {'registered': 1, 'confirmed': 1, 'present': 1},
 'capacity': 30,
 'entry_exit': [{'id': 'Entradas', 'data': [{'x': '10:00', 'y': 1}]},
  {'id': 'Salidas', 'data': [{'x': '12:00', 'y': 1}]}],
 'nationality': [{'country': 'País 4', 'userCount': 1}],
 'type': [{'id': 4, 'typeName': 'Sector X', 'typeCount': 1}]}

# CONEXIÓN DATABASES

In [32]:
from sqlalchemy import create_engine, inspect, text
from sqlalchemy.orm import sessionmaker
import pandas as pd
from datetime import timedelta
import locale
import os
from dotenv import load_dotenv

<br>

### AWS

In [33]:
load_dotenv()

AWS_USER = os.getenv('AWS_USER')
AWS_PASS = os.getenv('AWS_PASS')
AWS_HOST = os.getenv('AWS_HOST')
AWS_PORT = os.getenv('AWS_PORT')

In [34]:
db_url = f'mysql+mysqlconnector://{AWS_USER}:{AWS_PASS}@{AWS_HOST}:{AWS_PORT}/desafio'
aws_engine = create_engine(db_url)

In [35]:
inspector = inspect(aws_engine)
table_names = inspector.get_table_names()
table_names

['Amenities',
 'Contracted_services',
 'EducationalCenter',
 'EventUsers',
 'Events',
 'Facilities',
 'Feedback',
 'Guests',
 'Location_amenities',
 'Locations',
 'Organizations',
 'Roles',
 'SequelizeMeta',
 'Service_Provisions',
 'Service_companies',
 'Tokens',
 'Types',
 'Users']

<br>

### RENDER

In [36]:
load_dotenv()

RENDER_USER = os.getenv('RENDER_USER')
RENDER_PASS = os.getenv('RENDER_PASS')
RENDER_HOST = os.getenv('RENDER_HOST')
RENDER_PORT = os.getenv('RENDER_PORT')

In [37]:
db_url = f'postgresql://{RENDER_USER}:{RENDER_PASS}@{RENDER_HOST}:{RENDER_PORT}/desafio_dgp8'
render_engine = create_engine(db_url)

In [38]:
inspector = inspect(render_engine)
table_names = inspector.get_table_names()
table_names

['Types',
 'Contracted_services',
 'Tokens',
 'Roles',
 'Facilities',
 'Locations',
 'Service_companies',
 'EducationalCenter',
 'EventUsers',
 'Amenities',
 'Service_Provisions',
 'Organizations',
 'Feedback',
 'Location_amenities',
 'Events',
 'Guests',
 'Users']

<BR>

# QUERY-API DASHBOARD AWS

In [26]:
print(pd.to_datetime('23-03-01', yearfirst=True).day_name())
print(pd.to_datetime('23-03-01', yearfirst=True).month_name())

Wednesday
March


In [29]:
pd.read_sql_table('Events', con=aws_engine).columns

Index(['id', 'location_id', 'dateTime', 'duration_min', 'type', 'banner',
       'speacker', 'description', 'title', 'createdAt', 'updatedAt'],
      dtype='object')

In [30]:
evento = 'Evento 4'


query = f"""
SELECT
    E.dateTime,
    E.speacker,
    E.description,
    E.title,
    E.location_id,
    L.capacity,
    U.country,
    T.id,
    T.name,
    EU.arriveTime,
    EU.leaveTime,
    COUNT(DISTINCT EU.arriveTime),
    COUNT(DISTINCT EU.leaveTime),
    COUNT(DISTINCT T.name),
    COUNT(DISTINCT U.country)
FROM
    Events AS E
JOIN
    Locations AS L ON E.location_id = L.id
LEFT JOIN
    EventUsers AS EU ON E.id = EU.Event_id
LEFT JOIN
    Users AS U ON EU.User_id = U.id
LEFT JOIN
    Organizations AS O ON U.organization_id = O.id
LEFT JOIN
    Types AS T ON O.type_id = T.id
WHERE
    E.title = "{evento}";
"""



# Crea una sesión de SQLAlchemy
Session = sessionmaker(bind=aws_engine)
session = Session()

with session:
    # Ejecuta la consulta para leer la tabla Facilities
    database = session.execute(text(query))
    columnas = database.keys()

    # Recupera los resultados
    facilities_data = database.fetchall()

# Imprime los resultados
dict_table = {}
for row in facilities_data:
    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
    dict_table['title'] = row[3]
    dict_table['description'] = row[2]
    dict_table['speaker'] = row[1]
    dict_table['day'] = str(pd.to_datetime(row[0], yearfirst=True).strftime('%A')).capitalize()
    dict_table['month'] = str(pd.to_datetime(row[0], yearfirst=True).strftime('%B')).capitalize()
    dict_table['attendees'] = {'registered': [1 if row[9] is not None else 0][0], 
                               'confirmed': [1 if row[9] is not None else 0][0], 
                               'present': [1 if row[9] is not None else 0][0]}
    dict_table['capacity'] = row[5]
    dict_table['entry_exit'] = [{'id': 'Entradas', 'data': [{'x': [pd.to_datetime(row[9]).strftime('%H:%M') if row[9] != None else None], 'y': row[11]}]}, 
                                {'id': 'Salidas', 'data': [{'x': [pd.to_datetime(row[10]).strftime('%H:%M') if row[9] != None else None], 'y': row[12]}]}]
    dict_table['nationality'] = [{'country': row[6], 'userCount': row[14]}]
    dict_table['type'] = [{'id':row[7], 'typeName':row[8], 'typeCount':row[13]}]
    print(row)

dict_table

('2023-12-20', 'Sofia Pinilla', 'Intake Mayo23', 'Evento 4', 4, 30, None, None, None, None, None, 0, 0, 0, 0)


{'title': 'Evento 4',
 'description': 'Intake Mayo23',
 'speaker': 'Sofia Pinilla',
 'day': 'Miércoles',
 'month': 'Diciembre',
 'attendees': {'registered': 0, 'confirmed': 0, 'present': 0},
 'capacity': 30,
 'entry_exit': [{'id': 'Entradas', 'data': [{'x': [None], 'y': 0}]},
  {'id': 'Salidas', 'data': [{'x': [None], 'y': 0}]}],
 'nationality': [{'country': None, 'userCount': 0}],
 'type': [{'id': None, 'typeName': None, 'typeCount': 0}]}

In [28]:
from datetime import datetime

# evento = 'Planteamientos financieros y económicos'
# evento = 'Open Day Grado ADE'
# evento = 'Foro de Empleo IT'
evento = 'Conocer herramientas que mejoran la productividad de una empresa'


query = f"""
SELECT
    E.dateTime,
    E.speacker,
    E.description,
    E.title,
    E.location_id,
    L.capacity,
    U.country,
    O.name,
    T.name,
    EU.arriveTime,
    EU.leaveTime,
    COUNT(DISTINCT T.name),
    COUNT(DISTINCT U.country),
    U.confirmed
FROM
    Events AS E
JOIN
    Locations AS L ON E.location_id = L.id
LEFT JOIN
    EventUsers AS EU ON E.id = EU.event_id
LEFT JOIN
    Users AS U ON EU.user_id = U.id
LEFT JOIN
    Organizations AS O ON U.organization_id = O.id
LEFT JOIN
    Types AS T ON O.type_id = T.id
WHERE
    E.title = '{evento}'
GROUP BY
    E.dateTime,
    E.speacker,
    E.description,
    E.title,
    E.location_id,
    L.capacity,
    U.country,
    O.name,
    T.name,
    EU.arriveTime,
    EU.leaveTime,
    U.confirmed;
"""



# Crea una sesión de SQLAlchemy
Session = sessionmaker(bind=aws_engine)
session = Session()

with session:
    # Ejecuta la consulta para leer la tabla Facilities
    database = session.execute(text(query))
    columnas = database.keys()

    # Recupera los resultados
    facilities_data = database.fetchall()

# bucle for que corre todos los usuarios
dict_table = {}
horas_entrada = []
horas_salida = []
entry_exit_Entradas = {}
entry_exit_Salidas = {}
nationality = {}
types = {}
registered = 0
confirmed = 0
present = 0

for row in facilities_data:
    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
    dict_table['title'] = row[3]
    dict_table['description'] = row[2]
    dict_table['speaker'] = row[1]
    dict_table['day'] = str(pd.to_datetime(row[0], yearfirst=True).strftime('%A')).capitalize()
    dict_table['month'] = str(pd.to_datetime(row[0], yearfirst=True).strftime('%B')).capitalize()
    registered += int(row[11])
    confirmed += int(row[13])
    present += [1 if row[9] != None else 0][0]
    dict_table['capacity'] = row[5]

    # ENTRY_EXIT
    horas_entrada.append(row[9])
    horas_salida.append(row[10])
    
    horas_evento = []

    if row[9] == None:
        horas_evento.append(None)
    else:
        if row[10] == None:
            horas_evento.append(None)
        else:
            fecha_inicio = pd.to_datetime(row[9])
            fecha_fin = pd.to_datetime(row[10])
            hora_abierto = (fecha_inicio - timedelta(hours=1, minutes=5)).to_pydatetime()
            hora_cierre = (fecha_fin + timedelta(minutes=30)).to_pydatetime()
            while hora_abierto < hora_cierre:
                minutos_redondeados = (hora_abierto.minute // 5) * 5
                timestamp_redondeado = hora_abierto.replace(minute=minutos_redondeados, second=0)
                horas_evento.append(pd.to_datetime(timestamp_redondeado))
                hora_abierto += timedelta(minutes=5)

    for fecha in horas_evento:
        if fecha in horas_entrada:
            count = horas_entrada.count(fecha)
            if fecha not in entry_exit_Entradas.keys():
                entry_exit_Entradas[f'{fecha}'] = count
            else:
                entry_exit_Entradas[f'{fecha}'] = count

        if fecha in horas_salida:
            count = horas_salida.count(fecha)
            if fecha not in entry_exit_Salidas.keys():
                entry_exit_Salidas[f'{fecha}'] = count
            else:
                entry_exit_Salidas[f'{fecha}'] = count

    # NACIONALIDAD
    if row[6] not in nationality.keys():
        nationality[row[6]] = row[12]
    else:
        nationality[row[6]] += row[12]

    # TYPE
    if row[8] not in types.keys():
        types[row[8]] = {
            row[7]: row[11]
        }
    elif row[7] not in types[row[8]].keys():
        types[row[8]][row[7]] = row[11]
    else:
        types[row[8]][row[7]] += row[11]


dict_table['attendees'] = {'registered': registered, 
                           'confirmed': confirmed, 
                           'present': present}

list_types = []
for key, value in types.items():
    for company, count in value.items():
        list_types.append({'id':key, 'typeName':company, 'typeCount':count})

dict_table['type'] = list_types

dict_table['nationality'] = [{'country': country, 'userCount': count} for country, count in nationality.items()]

dict_table['entry_exit'] = [{'id': 'Entradas', 'data': [{'x': pd.to_datetime(fecha), 'y': count} if fecha != 'None' else {'x': fecha, 'y': count} for fecha, count in entry_exit_Entradas.items()]}, 
                            {'id': 'Salidas', 'data': [{'x': pd.to_datetime(fecha), 'y': count} if fecha != 'None' else {'x': fecha, 'y': count} for fecha, count in entry_exit_Salidas.items()]}]

dict_table

{'attendees': {'registered': 0, 'confirmed': 0, 'present': 0},
 'type': [],
 'nationality': [],
 'entry_exit': [{'id': 'Entradas', 'data': []}, {'id': 'Salidas', 'data': []}]}

<BR>

# QUERY-API DASHBOARD RENDER

In [39]:
from datetime import datetime

# evento = 'Planteamientos financieros y económicos'
# evento = 'Open Day Grado ADE'
# evento = 'Foro de Empleo IT'
evento = 'Conocer herramientas que mejoran la productividad de una empresa'


query = f"""
SELECT
    "E"."dateTime",
    "E"."speacker",
    "E"."description",
    "E"."title",
    "E"."location_id",
    "L"."capacity",
    "U"."country",
    "O"."name",
    "T"."name",
    "EU"."arriveTime",
    "EU"."leaveTime",
    COUNT(DISTINCT "T"."name"),
    COUNT(DISTINCT "U"."country"),
    "U"."confirmed"
FROM
    "Events" AS "E"
JOIN
    "Locations" AS "L" ON "E"."location_id" = "L"."id"
LEFT JOIN
    "EventUsers" AS "EU" ON "E"."id" = "EU"."event_id"
LEFT JOIN
    "Users" AS "U" ON "EU"."user_id" = "U"."id"
LEFT JOIN
    "Organizations" AS "O" ON "U"."organization_id" = "O"."id"
LEFT JOIN
    "Types" AS "T" ON "O"."type_id" = "T"."id"
WHERE
    "E"."title" = '{evento}'
GROUP BY
    "E"."dateTime",
    "E"."speacker",
    "E"."description",
    "E"."title",
    "E"."location_id",
    "L"."capacity",
    "U"."country",
    "O"."name",
    "T"."name",
    "EU"."arriveTime",
    "EU"."leaveTime",
    "U"."confirmed";
"""



# Crea una sesión de SQLAlchemy
Session = sessionmaker(bind=render_engine)
session = Session()

with session:
    # Ejecuta la consulta para leer la tabla Facilities
    database = session.execute(text(query))
    columnas = database.keys()

    # Recupera los resultados
    facilities_data = database.fetchall()

# bucle for que corre todos los usuarios
dict_table = {}
horas_entrada = []
horas_salida = []
entry_exit_Entradas = {}
entry_exit_Salidas = {}
nationality = {}
types = {}
registered = 0
confirmed = 0
present = 0

for row in facilities_data:
    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
    dict_table['title'] = row[3]
    dict_table['description'] = row[2]
    dict_table['speaker'] = row[1]
    dict_table['day'] = str(pd.to_datetime(row[0], yearfirst=True).strftime('%A')).capitalize()
    dict_table['month'] = str(pd.to_datetime(row[0], yearfirst=True).strftime('%B')).capitalize()
    registered += int(row[11])
    confirmed += int(row[13])
    present += [1 if row[9] != None else 0][0]
    dict_table['capacity'] = row[5]

    # ENTRY_EXIT
    horas_entrada.append(row[9])
    horas_salida.append(row[10])
    
    horas_evento = []

    if row[9] == None:
        horas_evento.append(None)
    else:
        if row[10] == None:
            horas_evento.append(None)
        else:
            fecha_inicio = pd.to_datetime(row[9])
            fecha_fin = pd.to_datetime(row[10])
            hora_abierto = (fecha_inicio - timedelta(hours=1, minutes=5)).to_pydatetime()
            hora_cierre = (fecha_fin + timedelta(minutes=30)).to_pydatetime()
            while hora_abierto < hora_cierre:
                minutos_redondeados = (hora_abierto.minute // 5) * 5
                timestamp_redondeado = hora_abierto.replace(minute=minutos_redondeados, second=0)
                horas_evento.append(pd.to_datetime(timestamp_redondeado))
                hora_abierto += timedelta(minutes=5)

    for fecha in horas_evento:
        if fecha in horas_entrada:
            count = horas_entrada.count(fecha)
            if fecha not in entry_exit_Entradas.keys():
                entry_exit_Entradas[f'{fecha}'] = count
            else:
                entry_exit_Entradas[f'{fecha}'] = count

        if fecha in horas_salida:
            count = horas_salida.count(fecha)
            if fecha not in entry_exit_Salidas.keys():
                entry_exit_Salidas[f'{fecha}'] = count
            else:
                entry_exit_Salidas[f'{fecha}'] = count

    # NACIONALIDAD
    if row[6] not in nationality.keys():
        nationality[row[6]] = row[12]
    else:
        nationality[row[6]] += row[12]

    # TYPE
    if row[8] not in types.keys():
        types[row[8]] = {
            row[7]: row[11]
        }
    elif row[7] not in types[row[8]].keys():
        types[row[8]][row[7]] = row[11]
    else:
        types[row[8]][row[7]] += row[11]


    print(row)


dict_table['attendees'] = {'registered': registered, 
                           'confirmed': confirmed, 
                           'present': present}

list_types = []
for key, value in types.items():
    for company, count in value.items():
        list_types.append({'id':key, 'typeName':company, 'typeCount':count})

dict_table['type'] = list_types

dict_table['nationality'] = [{'country': country, 'userCount': count} for country, count in nationality.items()]

dict_table['entry_exit'] = [{'id': 'Entradas', 'data': [{'x': pd.to_datetime(fecha), 'y': count} if fecha != 'None' else {'x': fecha, 'y': count} for fecha, count in entry_exit_Entradas.items()]}, 
                            {'id': 'Salidas', 'data': [{'x': pd.to_datetime(fecha), 'y': count} if fecha != 'None' else {'x': fecha, 'y': count} for fecha, count in entry_exit_Salidas.items()]}]

dict_table

(datetime.datetime(2023, 9, 1, 18, 0), 'Mar Pedroche', 'Mar Pedroche nos mostrará las aplicaciones y programas que te pueden ayudar a organizar mejor las tareas diarias en la oficina y aumentar la productividad.', 'Conocer herramientas que mejoran la productividad de una empresa', 5, 60, 'Croatia', 'Firmus', 'Miembros', datetime.datetime(2023, 9, 1, 18, 6), datetime.datetime(2023, 9, 1, 18, 43), 1, 1, '0')
(datetime.datetime(2023, 9, 1, 18, 0), 'Mar Pedroche', 'Mar Pedroche nos mostrará las aplicaciones y programas que te pueden ayudar a organizar mejor las tareas diarias en la oficina y aumentar la productividad.', 'Conocer herramientas que mejoran la productividad de una empresa', 5, 60, 'Cyprus', 'Amadeus IT', 'Otro', None, None, 1, 1, '0')
(datetime.datetime(2023, 9, 1, 18, 0), 'Mar Pedroche', 'Mar Pedroche nos mostrará las aplicaciones y programas que te pueden ayudar a organizar mejor las tareas diarias en la oficina y aumentar la productividad.', 'Conocer herramientas que mejora

{'title': 'Conocer herramientas que mejoran la productividad de una empresa',
 'description': 'Mar Pedroche nos mostrará las aplicaciones y programas que te pueden ayudar a organizar mejor las tareas diarias en la oficina y aumentar la productividad.',
 'speaker': 'Mar Pedroche',
 'day': 'Viernes',
 'month': 'Septiembre',
 'capacity': 60,
 'attendees': {'registered': 25, 'confirmed': 15, 'present': 21},
 'type': [{'id': 'Miembros', 'typeName': 'Firmus', 'typeCount': 1},
  {'id': 'Miembros', 'typeName': 'Implica-Corporate-Finance', 'typeCount': 1},
  {'id': 'Miembros', 'typeName': 'Ecija', 'typeCount': 1},
  {'id': 'Miembros', 'typeName': 'Grupo-Ribera', 'typeCount': 1},
  {'id': 'Miembros', 'typeName': 'Infraca', 'typeCount': 1},
  {'id': 'Miembros', 'typeName': 'Pinturas-Blatem', 'typeCount': 1},
  {'id': 'Miembros', 'typeName': 'BigBuy', 'typeCount': 1},
  {'id': 'Miembros', 'typeName': 'Grefusa', 'typeCount': 1},
  {'id': 'Miembros', 'typeName': 'RFL-Cargo', 'typeCount': 1},
  {'id'